# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

Submit "ratings.tsv" in Gradescope.

In [18]:
import pandas as pd

recipes = pd.read_csv('recipes.tsv', sep='\t')

ratings = pd.DataFrame()
ratings['recipe_slug'] = recipes['recipe_slug']

def rate_recipe(row):
    title = str(row.get('recipe_title', '')).lower()
    intro = str(row.get('recipe_introduction', '')).lower()
    text = title + ' ' + intro

    if ('bacon' in text and 'chocolate' in text) or \
       any(word in text for word in ['chocolate cake', 'chocolate souffle', 'brownies', 
                                      'chocolate babka', 'pain au chocolat', 'chocolate croissant']):
        return 5

    if 'bacon' in text or ('peanut butter' in text and 'chocolate' in text):
        return 4

    if any(word in text for word in ['pasta', 'lasagna', 'taco', 'burrito', 'enchilada', 
                                      'nacho', 'quesadilla', 'noodle', 'crisp', 'crumble']):
        return 3

    if any(word in text for word in ['quiche', 'soup', 'salad', 'pickled', 'relish']):
        return 2

    if any(word in text for word in ['oyster', 'spam', 'falafel', 'asparagus', 
                                      'mushroom', 'spinach', 'vegetarian', 'cold noodles']):
        return 1

    return 3

ratings['rating_raw'] = recipes.apply(rate_recipe, axis=1)

ratings['rating'] = (ratings['rating_raw'] - 1) * 0.25

num_zeros = (ratings['rating'] == 0.0).sum()
if num_zeros < 10:
    needed = 10 - num_zeros
    candidates = ratings[ratings['rating'] > 0.0].head(needed).index
    ratings.loc[candidates, 'rating'] = 0.0

num_ones = (ratings['rating'] == 1.0).sum()
if num_ones < 10:
    needed = 10 - num_ones
    candidates = ratings[ratings['rating'] < 1.0].tail(needed).index
    ratings.loc[candidates, 'rating'] = 1.0

ratings = ratings[['recipe_slug', 'rating']]

ratings.to_csv('ratings.tsv', sep='\t', index=False)

print(f"Created ratings.tsv with {len(ratings)} recipes")
print(f"Ratings of 0.0: {(ratings['rating'] == 0.0).sum()}")
print(f"Ratings of 1.0: {(ratings['rating'] == 1.0).sum()}")
print(f"\nDistribution:")
print(ratings['rating'].value_counts().sort_index())


Created ratings.tsv with 100 recipes
Ratings of 0.0: 10
Ratings of 1.0: 10

Distribution:
rating
0.00    10
0.25     6
0.50    60
0.75    14
1.00    10
Name: count, dtype: int64


## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [19]:
# YOUR CHANGES HERE

import pandas as pd

ratings = pd.read_csv('ratings.tsv', sep='\t')
recipe_tags = pd.read_csv('recipe-tags.tsv', sep='\t')

features = recipe_tags.pivot_table(
    index='recipe_slug',
    columns='recipe_tag',
    aggfunc=len,
    fill_value=0
).reset_index()

tag_columns = [col for col in features.columns if col != 'recipe_slug']
for col in tag_columns:
    features[col] = (features[col] > 0).astype(int)

features.insert(1, 'bias', 1)

all_recipes = pd.DataFrame({'recipe_slug': ratings['recipe_slug']})
features = all_recipes.merge(features, on='recipe_slug', how='left')

features = features.fillna(0)

for col in features.columns:
    if col not in ['recipe_slug', 'bias']:
        features[col] = features[col].astype(int)

features = features.set_index('recipe_slug').loc[ratings['recipe_slug']].reset_index()

features.to_csv('features.tsv', sep='\t', index=False)

print(f"Created features.tsv with {len(features)} recipes")
print(f"Number of tag columns: {len(features.columns) - 2}")
print(f"\nFirst few columns: {list(features.columns[:5])}")
print(f"\nVerifying order matches ratings.tsv: {(features['recipe_slug'] == ratings['recipe_slug']).all()}")

Created features.tsv with 100 recipes
Number of tag columns: 296

First few columns: ['recipe_slug', 'bias', 'alfredo', 'almond', 'american']

Verifying order matches ratings.tsv: True


/tmp/ipykernel_5662/2815755130.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features.insert(1, 'bias', 1)


Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [20]:
# YOUR CHANGES HERE

import pandas as pd
from sklearn.linear_model import Ridge

features = pd.read_csv('features.tsv', sep='\t')
ratings = pd.read_csv('ratings.tsv', sep='\t')

assert (features['recipe_slug'] == ratings['recipe_slug']).all(), "Recipe slugs must be aligned!"

X = features.drop(columns='recipe_slug').values 
y = ratings['rating'].values

ridge_model = Ridge(alpha=1.0, fit_intercept=False)
ridge_model.fit(X, y)

coef = ridge_model.coef_
print("Ridge coefficients (including bias):")
print(coef)

y_pred = ridge_model.predict(X)
print("\nFirst 10 predicted ratings:")
print(y_pred[:10])


Ridge coefficients (including bias):
[ 3.61673203e-01 -4.54731138e-02  3.70314126e-02 -7.87947834e-03
  9.79271317e-03 -4.37892450e-03 -8.89362289e-02  3.80169129e-02
  3.91288792e-02  1.90180516e-02  1.46451091e-01 -2.20809792e-03
 -2.53883905e-02 -3.70824971e-02  6.93697879e-02  1.91052055e-02
  2.40566415e-02  2.30902078e-03  3.59702811e-02  2.30902078e-03
 -5.96404279e-05 -1.28361183e-03  5.21810876e-03  5.21810876e-03
  2.03594366e-02 -3.74817142e-02  4.92922944e-02  5.55725769e-02
  5.16789855e-02 -8.92865042e-02  3.57470053e-02  1.22780668e-01
 -2.78592418e-03  4.26428391e-02  3.07640450e-02  4.08432745e-02
  6.08548757e-02  5.64721657e-02  5.21810876e-03  5.37232545e-04
  1.10964360e-02  2.02237958e-02  1.08046456e-01 -2.17247042e-02
 -3.78511530e-02  3.07640450e-02  1.32502412e-02  2.30902078e-03
  2.60300244e-02  1.33098816e-02  1.29302592e-02  1.69099634e-01
  7.12716094e-02  5.16789855e-02 -2.02396649e-03  1.81450706e-02
  3.06713280e-02 -4.26786499e-02  4.40870021e-02  4.2

Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not add anything for the `intercept_` attribute of a scikit-learn model; this will be covered by the coefficient for the bias column added in part 2.

In [21]:
# YOUR CHANGES HERE

import pandas as pd
from sklearn.linear_model import Ridge

features = pd.read_csv('features.tsv', sep='\t')
ratings = pd.read_csv('ratings.tsv', sep='\t')

assert (features['recipe_slug'] == ratings['recipe_slug']).all(), "Recipe slugs must be aligned!"

X = features.drop(columns='recipe_slug').values
y = ratings['rating'].values

ridge_model = Ridge(alpha=1.0, fit_intercept=False)
ridge_model.fit(X, y)

feature_names = features.drop(columns='recipe_slug').columns

coef_df = pd.DataFrame({
    'recipe_tag': feature_names,
    'coefficient': ridge_model.coef_
})

coef_df.to_csv('model.tsv', sep='\t', index=False)

print(f"Saved ridge coefficients to model.tsv with {len(coef_df)} rows")
print(coef_df.head())


Saved ridge coefficients to model.tsv with 297 rows
  recipe_tag  coefficient
0       bias     0.361673
1    alfredo    -0.045473
2     almond     0.037031
3   american    -0.007879
4  appetizer     0.009793


Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [22]:
# YOUR CHANGES HERE


features = pd.read_csv('features.tsv', sep='\t')
model = pd.read_csv('model.tsv', sep='\t')

X = features.drop(columns='recipe_slug').values

coefficients = model['coefficient'].values

score_estimates = X @ coefficients

estimates = pd.DataFrame({
    'recipe_slug': features['recipe_slug'],
    'score_estimate': score_estimates
})

estimates.to_csv('estimates.tsv', sep='\t', index=False)

print(f"Saved score estimates to estimates.tsv with {len(estimates)} recipes")
print(f"\nFirst 10 estimates:")
print(estimates.head(10))
print(f"\nEstimate statistics:")
print(estimates['score_estimate'].describe())

Saved score estimates to estimates.tsv with 100 recipes

First 10 estimates:
                    recipe_slug  score_estimate
0                       falafel        0.097210
1                    spamburger        0.040405
2              bacon-fried-rice        0.054564
3               chicken-fingers        0.091347
4                   apple-crisp        0.090382
5         cranberry-apple-crisp        0.034435
6  bacon-chocolate-chip-cookies        0.283303
7                        sujebi        0.044349
8               pasta-primavera        0.146696
9                         ramen        0.494782

Estimate statistics:
count    100.000000
mean       0.516383
std        0.209628
min        0.034435
25%        0.473171
50%        0.497139
75%        0.575325
max        0.979641
Name: score_estimate, dtype: float64


Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [23]:
# YOUR CHANGES HERE

import numpy as np

features = pd.read_csv('features.tsv', sep='\t')
ratings = pd.read_csv('ratings.tsv', sep='\t')
model = pd.read_csv('model.tsv', sep='\t')

assert (features['recipe_slug'] == ratings['recipe_slug']).all(), "Recipe slugs must be aligned!"

X = features.drop(columns='recipe_slug').values
y = ratings['rating'].values

ridge_lambda = 1.0

linucb_alpha = 2.0

d = X.shape[1]

A = X.T @ X + ridge_lambda * np.eye(d)

A_inv = np.linalg.inv(A)

theta = A_inv @ X.T @ y

score_bounds = []

for i in range(len(X)):
    x_i = X[i]
    
    estimate = x_i @ theta
    
    uncertainty = linucb_alpha * np.sqrt(x_i @ A_inv @ x_i)
    
    upper_bound = estimate + uncertainty
    score_bounds.append(upper_bound)

bounds = pd.DataFrame({
    'recipe_slug': features['recipe_slug'],
    'score_bound': score_bounds
})

bounds.to_csv('bounds.tsv', sep='\t', index=False)

print(f"Saved LinUCB upper bounds to bounds.tsv with {len(bounds)} recipes")
print(f"\nFirst 10 bounds:")
print(bounds.head(10))
print(f"\nBound statistics:")
print(bounds['score_bound'].describe())

Saved LinUCB upper bounds to bounds.tsv with 100 recipes

First 10 bounds:
                    recipe_slug  score_bound
0                       falafel     1.863465
1                    spamburger     1.934742
2              bacon-fried-rice     1.920721
3               chicken-fingers     1.885462
4                   apple-crisp     1.866724
5         cranberry-apple-crisp     1.860560
6  bacon-chocolate-chip-cookies     1.971459
7                        sujebi     1.869244
8               pasta-primavera     1.831731
9                         ramen     2.405777

Bound statistics:
count    100.000000
mean       2.279328
std        0.207047
min        1.831731
25%        2.160796
50%        2.284940
75%        2.369329
max        2.837011
Name: score_bound, dtype: float64


Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

In [24]:
# YOUR CHANGES HERE


features = pd.read_csv('features.tsv', sep='\t')
ratings = pd.read_csv('ratings.tsv', sep='\t')

assert (features['recipe_slug'] == ratings['recipe_slug']).all(), "Recipe slugs must be aligned!"

X = features.drop(columns='recipe_slug').values
y = ratings['rating'].values
recipe_slugs = features['recipe_slug'].values

ridge_lambda = 1.0
linucb_alpha = 2.0
n_recommendations = 100
d = X.shape[1] 

A = ridge_lambda * np.eye(d)  
b = np.zeros(d)  # b = 0

recommendations = []
recommended_indices = set()

for t in range(n_recommendations):
    A_inv = np.linalg.inv(A)
    
    theta = A_inv @ b
    
    best_ucb = -np.inf
    best_idx = None
    
    for i in range(len(X)):
        if i in recommended_indices:
            continue
        
        x_i = X[i]
        estimate = x_i @ theta
        uncertainty = linucb_alpha * np.sqrt(x_i @ A_inv @ x_i)
        ucb = estimate + uncertainty
        
        if ucb > best_ucb:
            best_ucb = ucb
            best_idx = i
    
    recommended_indices.add(best_idx)
    x_chosen = X[best_idx]
    reward = y[best_idx]
    
    recommendations.append({
        'recipe_slug': recipe_slugs[best_idx],
        'score_bound': best_ucb,
        'reward': reward
    })
    
    A += np.outer(x_chosen, x_chosen)  
    b += reward * x_chosen  
    
    if (t + 1) % 10 == 0:
        print(f"Made {t + 1} recommendations...")

recommendations_df = pd.DataFrame(recommendations)

recommendations_df.to_csv('recommendations.tsv', sep='\t', index=False)

print(f"\nSaved {len(recommendations_df)} recommendations to recommendations.tsv")
print(f"\nFirst 10 recommendations:")
print(recommendations_df.head(10))
print(f"\nReward statistics:")
print(recommendations_df['reward'].describe())
print(f"\nCumulative reward: {recommendations_df['reward'].sum():.2f}")
print(f"Average reward: {recommendations_df['reward'].mean():.4f}")

Made 10 recommendations...
Made 20 recommendations...
Made 30 recommendations...
Made 40 recommendations...
Made 50 recommendations...
Made 60 recommendations...
Made 70 recommendations...
Made 80 recommendations...
Made 90 recommendations...
Made 100 recommendations...

Saved 100 recommendations to recommendations.tsv

First 10 recommendations:
        recipe_slug  score_bound  reward
0     apple-crumble     7.483315     0.5
1     ma-la-chicken     7.225922     0.5
2       quesadillas     7.215690     0.5
3             ramen     7.205736     0.5
4  pain-au-chocolat     6.948626     1.0
5   chocolate-babka     6.998701     1.0
6        spamburger     7.010668     0.0
7  bacon-fried-rice     6.981256     0.0
8       nacho-fries     6.732809     0.5
9   cranberry-sauce     6.672025     0.5

Reward statistics:
count    100.000000
mean       0.520000
std        0.250454
min        0.000000
25%        0.500000
50%        0.500000
75%        0.500000
max        1.000000
Name: reward, dtype: 

Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.